In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Forecasted Data**

In [2]:
import pandas as pd
import glob
import os

folder_path = "/content/drive/MyDrive/GRAD"

csv_files = glob.glob(os.path.join(folder_path, "*forecast*.csv"))

forecast = pd.concat(
    (pd.read_csv(f, index_col=0).assign(filename=os.path.basename(f)) for f in csv_files),
    ignore_index=True
)

forecast.head()

,Date,Lower Bound,Upper Bound,Point Estimation,filename
0,2025Q3,47027.039552,52134.704685,49580.872118,alexandria-compounds-forecast (1).csv
1,2025Q4,46404.882686,51118.916849,48761.899767,alexandria-compounds-forecast (1).csv
2,2026Q1,45613.547261,50989.421983,48301.484622,alexandria-compounds-forecast (1).csv
3,2026Q2,46806.515189,52017.657724,49412.086457,alexandria-compounds-forecast (1).csv
4,2026Q3,48020.143289,54263.069292,51141.606291,alexandria-compounds-forecast (1).csv


In [18]:
forecast.to_csv("Forecasted_prices.csv")

In [3]:
import re
# Clean the filename column
def clean_filename(name):
    # Remove forecast, .csv, trailing numbers like (1), and dashes
    name = re.sub(r'forecast', '', name, flags=re.IGNORECASE)
    name = re.sub(r'\(\d+\)', '', name)
    name = re.sub(r'\.csv', '', name, flags=re.IGNORECASE)
    name = name.replace('-', ' ')
    return name.strip()

forecast["filename"] = forecast['filename'].apply(clean_filename)
forecast = forecast[forecast["filename"] != "alexandria compounds"]
forecast = forecast.rename(columns={"filename": "Area"})
forecast

,Date,Lower Bound,Upper Bound,Point Estimation,Area
12,2025Q3,4867.370365,7317.183512,6092.276939,hay al agami
13,2025Q4,4901.225381,7299.047521,6100.136451,hay al agami
14,2026Q1,9013.453279,10360.252657,9686.852968,hay al agami
15,2026Q2,7909.156609,10856.223968,9382.690289,hay al agami
16,2026Q3,8378.536398,11041.190571,9709.863484,hay al agami
...,...,...,...,...,...
91,2027Q2,24994.807487,29222.110522,27108.459004,hay wasat
92,2027Q3,24858.752204,30171.891458,27515.321831,hay wasat
93,2027Q4,24929.160472,30502.459298,27715.809885,hay wasat
94,2028Q1,31349.540127,38204.194285,34776.867206,hay wasat


**Updated Timeseries**

In [4]:
folder_path = "/content/drive/MyDrive/GRAD/updated_timeseries"

csv_files = glob.glob(os.path.join(folder_path, "*.csv"))

df = pd.concat(
    (pd.read_csv(f, index_col=0) for f in csv_files),
    ignore_index=True
)

df = df[df["Bedrooms No."] == 3]

df_cleaned = df.drop(columns=['Bedrooms No.'])

for col in df_cleaned.columns[1:]:
    df_cleaned[col] = (
        df_cleaned[col]
        .astype(str)
        .str.replace('EGP', '', regex=False)
        .str.replace(',', '', regex=False)
        .astype(float)
    )
year_2025 = df_cleaned[["2025", "Area"]]
year_2025 = year_2025.groupby('Area').mean(numeric_only=True).reset_index()

year_2025 = year_2025.T.reset_index(drop=True)
year_2025.columns = year_2025.iloc[0]
year_2025 = year_2025[1:].reset_index(drop=True)
year_2025["Date"] = "2025Q1"

year_2025

,hay-al-agami,hay-al-amereyah,hay-awal-el-montazah,hay-sharq,hay-than-el-montazah,hay-wasat,Date
0,12273.0,17438.0,18440.0,29778.0,20784.0,26858.0,2025Q1


In [5]:
year_2025.to_csv("2025Q1_prices.csv")

**Last Prices**

In [6]:
histo_prices = pd.read_csv("/content/drive/MyDrive/GRAD/ppm_2017_2024_areas_final_v8.csv", index_col=0)
histo_prices = histo_prices[histo_prices["Date"] == "2024Q4"]
histo_prices

,Date,hay-al-agami,alexandria-compounds,hay-gharb,hay-al-amereyah,hay-wasat,hay-than-el-montazah,hay-el-gomrok,hay-sharq,hay-awal-el-montazah
31,2024Q4,9502.0,34842.0,17164.0,15270.375,22490.25,15984.666667,24826.0,19399.5,14774.0


In [7]:
histo_prices.loc[histo_prices['Date'] == '2024Q4', 'hay-gharb'].values[0]

np.float64(17164.0)

In [8]:
# histo_prices.reset_index().to_dict()["hay-gharb"][0]
histo_prices = histo_prices.reset_index().to_dict()

In [9]:
forecast

,Date,Lower Bound,Upper Bound,Point Estimation,Area
12,2025Q3,4867.370365,7317.183512,6092.276939,hay al agami
13,2025Q4,4901.225381,7299.047521,6100.136451,hay al agami
14,2026Q1,9013.453279,10360.252657,9686.852968,hay al agami
15,2026Q2,7909.156609,10856.223968,9382.690289,hay al agami
16,2026Q3,8378.536398,11041.190571,9709.863484,hay al agami
...,...,...,...,...,...
91,2027Q2,24994.807487,29222.110522,27108.459004,hay wasat
92,2027Q3,24858.752204,30171.891458,27515.321831,hay wasat
93,2027Q4,24929.160472,30502.459298,27715.809885,hay wasat
94,2028Q1,31349.540127,38204.194285,34776.867206,hay wasat


In [10]:
forecast = forecast.rename(columns={"Lower Bound": "Lower_Bound", "Upper Bound": "Upper_Bound", "Point Estimation": "Point_Estimation"})

In [11]:
forecast["Area"] = forecast["Area"].str.replace(" ", "-")

In [12]:
LB_Percent_Change = []
UB_Percent_Change = []
for row in forecast.itertuples(index=True):
    lower_bound_change = (row.Lower_Bound - histo_prices[row.Area][0]) / histo_prices[row.Area][0]
    upper_bound_change = (row.Upper_Bound - histo_prices[row.Area][0]) / histo_prices[row.Area][0]

    lower_bound_change = round(lower_bound_change * 100, 4)
    upper_bound_change = round(upper_bound_change * 100, 4)

    LB_Percent_Change.append(lower_bound_change)
    UB_Percent_Change.append(upper_bound_change)

forecast = forecast.assign(LB_Percent_Change=LB_Percent_Change, UB_Percent_Change=UB_Percent_Change)
forecast

,Date,Lower_Bound,Upper_Bound,Point_Estimation,Area,LB_Percent_Change,UB_Percent_Change
12,2025Q3,4867.370365,7317.183512,6092.276939,hay-al-agami,-48.7753,-22.9932
13,2025Q4,4901.225381,7299.047521,6100.136451,hay-al-agami,-48.4190,-23.1841
14,2026Q1,9013.453279,10360.252657,9686.852968,hay-al-agami,-5.1415,9.0323
15,2026Q2,7909.156609,10856.223968,9382.690289,hay-al-agami,-16.7632,14.2520
16,2026Q3,8378.536398,11041.190571,9709.863484,hay-al-agami,-11.8234,16.1986
...,...,...,...,...,...,...,...
91,2027Q2,24994.807487,29222.110522,27108.459004,hay-wasat,11.1362,29.9324
92,2027Q3,24858.752204,30171.891458,27515.321831,hay-wasat,10.5312,34.1554
93,2027Q4,24929.160472,30502.459298,27715.809885,hay-wasat,10.8443,35.6253
94,2028Q1,31349.540127,38204.194285,34776.867206,hay-wasat,39.3917,69.8700


In [13]:
forecast.head(15)

,Date,Lower_Bound,Upper_Bound,Point_Estimation,Area,LB_Percent_Change,UB_Percent_Change
12,2025Q3,4867.370365,7317.183512,6092.276939,hay-al-agami,-48.7753,-22.9932
13,2025Q4,4901.225381,7299.047521,6100.136451,hay-al-agami,-48.4190,-23.1841
14,2026Q1,9013.453279,10360.252657,9686.852968,hay-al-agami,-5.1415,9.0323
15,2026Q2,7909.156609,10856.223968,9382.690289,hay-al-agami,-16.7632,14.2520
16,2026Q3,8378.536398,11041.190571,9709.863484,hay-al-agami,-11.8234,16.1986
17,2026Q4,8405.985539,11074.456673,9740.221106,hay-al-agami,-11.5346,16.5487
18,2027Q1,12478.796425,14622.968758,13550.882591,hay-al-agami,31.3281,53.8936
19,2027Q2,12310.956383,14494.462375,13402.709379,hay-al-agami,29.5617,52.5412
20,2027Q3,12422.282163,14494.605696,13458.443930,hay-al-agami,30.7333,52.5427
21,2027Q4,12426.074373,14506.542240,13466.308306,hay-al-agami,30.7733,52.6683


In [14]:
histo_prices["hay-al-agami"][0]

9502.0

In [15]:
top_change = forecast.loc[forecast.groupby('Area')['UB_Percent_Change'].idxmax()]

In [16]:
new_columns_order = ['Area', 'Date', 'Point_Estimation', 'Lower_Bound', 'Upper_Bound', 'LB_Percent_Change', 'UB_Percent_Change']
top_change = top_change[new_columns_order]
top_change

,Area,Date,Point_Estimation,Lower_Bound,Upper_Bound,LB_Percent_Change,UB_Percent_Change
23,hay-al-agami,2028Q2,16133.541555,14869.850187,17397.232924,56.4918,83.0902
35,hay-al-amereyah,2028Q2,26382.894405,24730.642072,28035.146739,61.9518,83.5917
46,hay-awal-el-montazah,2028Q1,24871.468478,23631.983965,26110.952991,59.9566,76.7358
58,hay-gharb,2028Q1,32816.669169,31222.702369,34410.635969,81.9081,100.4814
70,hay-sharq,2028Q1,31477.087255,30161.656601,32792.517908,55.4765,69.0380
83,hay-than-el-montazah,2028Q2,26920.466598,25790.571259,28050.361937,61.3457,75.4829
94,hay-wasat,2028Q1,34776.867206,31349.540127,38204.194285,39.3917,69.8700
